In [1]:
from visual_behavior import database as db
from visual_behavior import utilities as vbu

from allensdk.brain_observatory.behavior import sync
from allensdk.brain_observatory.behavior.behavior_project_cache import BehaviorProjectCache as bpc

/home/dougo/.local/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)
/home/dougo/Code/AllenSDK/allensdk/brain_observatory/behavior/behavior_ophys_api/behavior_ophys_nwb_api.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
def get_path(well_known_files, name):
    '''
    get path from well_known_files dataframe
    inputs:
        well_known_files (dataframe with index set to name)
        name (string with desired name)
    returns:
        string with full filepath
    '''
    if name in well_known_files.index:
        return ''.join(well_known_files.loc[name][['storage_directory', 'filename']].tolist())
    else:
        return None
    

def get_movie_frame_count(osid, movie_type):
    '''
    get movie frame count for a given movie type
    inputs:
        ophys_session_id
        movie_type ('behavior_monitoring' or 'eye_tracking')
    returns:
        number of frames in the given movie
    '''
    well_known_files = db.get_well_known_files(osid).set_index('name')
    if movie_type == 'eye_tracking':
        movie_path = get_path(well_known_files, 'RawEyeTrackingVideo')
    elif movie_type == 'behavior_monitoring':
        movie_path = get_path(well_known_files, 'RawBehaviorTrackingVideo')
    if movie_path:
        movie = vbu.Movie(movie_path)
        return movie.frame_count
    else:
        return None

## for ophys session 995558073, here are the behavior and eye movie frame counts:

In [3]:
ophys_session_id = 995558073
for movie_type in ['behavior_monitoring','eye_tracking']:
    print('number of frames in {} movie: {}'.format(
        movie_type, 
        get_movie_frame_count(ophys_session_id, movie_type)
    ))

number of frames in behavior_monitoring movie: 137046
number of frames in eye_tracking movie: 137058


## using the `behavior_monitoring` and `eye_tracking` keys directly from the AllenSDK `get_sync_data` function returns *reversed* counts for the sync lines:

In [4]:
well_known_files = db.get_well_known_files(ophys_session_id).set_index('name')
sync_path = get_path(well_known_files, 'OphysRigSync')
sync_data = sync.get_sync_data(sync_path)

print('sync_path: {}\n'.format(sync_path))

for movie_type in ['behavior_monitoring','eye_tracking']:
    print('number of rising edges in {} sync line: {}'.format(
        movie_type,
        len(sync_data[movie_type])
    ))

sync_path: /allen/programs/braintv/production/visualbehavior/prod4/specimen_962177507/ophys_session_995558073/995558073_sync.h5

number of rising edges in behavior_monitoring sync line: 137049
number of rising edges in eye_tracking sync line: 137058
